# Notebook de validation du pipeline RAG

Dans le présent notebook, nous allons:

    - tester plusieurs requêtes en boucle,
    - afficher les chunks utilisés,
    - annoter manuellement la pertinence des réponses,
    - stocker les résultats pour analyse ou feedback d’équipe.
    - utiliser un frameworks spécialisé pour évaluer le système RAG => RAGAS

In [1]:
# Imports et chargement du vectorstore + LLM

from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()
embedding_function = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
vectorstore = FAISS.load_local("../data/embeddings/faiss_store", embedding_function, allow_dangerous_deserialization=True)
llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-4o", temperature=0.7)

c:\Users\aheij\Desktop\Jedha\Data Full Stack\Projects for Submission\06 Final Project bloc 6\projet-culture-g\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\aheij\AppData\Local\Temp\ipykernel_17640\162009438.py:11: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
C:\Users\aheij\AppData\Local\Temp\ipykernel_17640\162009438.py:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10

In [2]:
# Fonction pour construire le prompt avec le contexte et la question

def build_prompt(context, question):
    return f"""
Tu es *Complice*, un assistant bienveillant qui aide les adolescents autistes à mieux comprendre les émotions, les relations sociales et les situations du quotidien.

Voici des extraits de documents pertinents :
{context}

Question : {question}

Réponds en français, avec clarté, douceur et concision. Utilise un ton rassurant, inclusif et jamais culpabilisant. Si la réponse est incertaine, propose des pistes ou invite à en discuter avec un adulte de confiance.
"""


In [3]:
# Fonction RAG avec affichage des chunks utilisés

def rag_verbose(question, k=5):
    docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    prompt = build_prompt(context, question)
    response = llm.invoke(prompt)

    return {
        "question": question,
        "response": response.content,
        "chunks": [doc.page_content for doc in docs],
        "sources": [doc.metadata.get("source", "inconnu") for doc in docs]
    }


In [4]:
# Boucle de test avec annotation manuelle

questions = [
    "Pourquoi certaines personnes sont autistes ?",
    "Est-ce que l'autisme se guérit ?",
    "Comment savoir si je suis autiste ?",
    "Est-ce ma faute si je suis autiste ?",
    "Comment gérer mes émotions quand je suis en colère ?",
    "Comment faire des amis quand on est autiste ?",
    "Pourquoi j'ai du mal à comprendre les expressions faciales des autres ?",
    "Comment expliquer à mes amis que je suis autiste ?",
    "Que faire si je me sens submergé par les bruits ou les lumières ?",
    "Est-ce que je suis fou ?",
    "Comment gérer l'anxiété et le stress au lycée ?",
    "Comment faire pour mieux dormir la nuit ?",
    "Pourquoi j'ai du mal à me faire des amis ?",
    "Comment faire pour surmonter ma peur des autres ?",
    "Est-ce que je suis bizarre parce que je suis autiste ?",
    "Comment faire pour mieux m'organiser au quotidien ?",
    "Pourquoi j'ai du mal à comprendre les blagues et l'humour des autres ?",
    "Comment faire pour surmonter le stress face aux contrôlesau lycée ?",
]

results = []

for q in questions:
    output = rag_verbose(q)
    print(f"\n🟦 Question : {output['question']}")
    print(f"🧠 Réponse : {output['response']}\n")
    for i, chunk in enumerate(output['chunks'], 1):
        print(f"🔹 Chunk {i} — Source : {output['sources'][i-1]}")
        print(chunk[:300] + "...\n")

    # Annotation manuelle
    rating = input("Pertinence de la réponse (0 = hors sujet, 1 = partielle, 2 = pertinente, 3 = excellente) : ")
    output["rating"] = int(rating)
    results.append(output)



🟦 Question : Pourquoi certaines personnes sont autistes ?
🧠 Réponse : Salut ! C'est une question très intéressante et importante que tu poses. La vérité, c'est que les scientifiques ne savent pas encore exactement pourquoi certaines personnes sont autistes et d'autres non. Cependant, il y a plusieurs pistes qui sont explorées.

D'abord, la génétique joue un rôle important. Cela signifie que l'autisme peut être en partie lié aux gènes que nous héritons de nos parents. Il est courant qu'un enfant autiste ait un parent ou un grand-parent qui présente des caractéristiques similaires.

Il y a aussi des recherches qui examinent le rôle du système immunitaire et des anomalies dans le cerveau. Par exemple, certaines études ont observé que la taille et le fonctionnement de certaines parties du cerveau peuvent être différents chez les personnes autistes.

Cependant, tout cela reste complexe et les scientifiques continuent d'étudier pour mieux comprendre. Ce qui est important, c'est de savoir qu

In [ ]:
# Sauvegarde des résultats
df = pd.DataFrame(results)
df.to_csv("../data/validation_results.csv", index=False)


In [13]:
df.head(20)

,question,response,chunks,sources,rating
0,Pourquoi certaines personnes sont autistes ?,Salut ! C'est une question très intéressante e...,[enfants de mères exposées au virus de la rubé...,"[Comprendre_lautisme_pour_les_nuls, Comprendre...",2
1,Est-ce que l'autisme se guérit ?,L'autisme n'est pas quelque chose que l'on peu...,[Utiliser les méthodes naturelles et biomédica...,"[Comprendre_lautisme_pour_les_nuls, Comprendre...",2
2,Comment savoir si je suis autiste ?,Savoir si tu es autiste peut être un chemin pe...,"[Es-tu autiste, toi aussi ? Cherches-tu à mieu...","[Etre autiste et ado_ Strategies - Inconnu(e),...",2
3,Est-ce ma faute si je suis autiste ?,"Non, ce n'est absolument pas ta faute si tu es...","[Es-tu autiste, toi aussi ? Cherches-tu à mieu...","[Etre autiste et ado_ Strategies - Inconnu(e),...",3
4,Comment gérer mes émotions quand je suis en co...,"Gérer ses émotions, surtout la colère, peut êt...",[suivante vous donne d’autres astuces pour sur...,"[Comprendre_lautisme_pour_les_nuls, 100 idees ...",2
5,Comment faire des amis quand on est autiste ?,Faire des amis quand on est autiste peut sembl...,[TOP 10 DES STRATÉGIES D’AMITIÉ Choisis le typ...,"[Etre autiste et ado_ Strategies - Inconnu(e),...",3
6,Pourquoi j'ai du mal à comprendre les expressi...,C'est tout à fait normal d'avoir du mal à comp...,"[De même, j’ai remarqué que certains enfants a...",[683956044-Le-Syndrome-dAsperger-Guide-Complet...,2
7,Comment expliquer à mes amis que je suis autis...,Expliquer à tes amis que tu es autiste peut se...,[Vous n’avez pas à inventer des explications p...,"[Comprendre_lautisme_pour_les_nuls, Comprendre...",2
8,Que faire si je me sens submergé par les bruit...,Lorsque tu te sens submergé par les bruits ou ...,[341 Sensibilité sensorielle Cela peut mener à...,[683956044-Le-Syndrome-dAsperger-Guide-Complet...,2
9,Est-ce que je suis fou ?,"Salut,\n\nTu n'es pas fou. Se sentir différent...",[On m’a prise pour une extraterrestre et un ph...,[683956044-Le-Syndrome-dAsperger-Guide-Complet...,2


# Utilisation de RAGAS (outil conçu pour évaluer les systèmes RAG avec des métriques précises)

L’intégration de RAGAS dans ce notebook de validation est une manière de calibrer finement les réponses de Complice, surtout dans un contexte aussi délicat et humain: l'outil final de ce projet s'adresse à de jeunes personnes vulnérables donc il est important de calibrer les réponses de manière à ce qu'elles soient pertinentes et bienveillantes.

In [6]:
# Installation de RAGAS pour évaluation avancée

!pip install ragas

  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.7 MB 1.5 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.7 MB 987.4 kB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.7 MB 1.3 MB/s eta 0:00:02
   -------------------------- ------------- 1.8/2.7 MB 1.7 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.7 MB 1.7 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 1.3 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 1.5 MB/s eta 0:00:00
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)


  You can safely remove it manually.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Préparer les données au format RAGAS, c'est à dire un dataset structuré avec les champs suivants :

    - question — la requête utilisateur

    - ground_truth — la réponse attendue (si disponible)

    - contexts — les chunks récupérés

    - answer — la réponse générée par le LLM

    - metadata — optionnel, infos sur les sources
    
Nous allons construire ce dataset à partir de la boucle de validation effectué ci-avant.

In [14]:
df = df.rename(columns={"response": "answer"})

In [17]:
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness, answer_relevancy
from datasets import Dataset  # Hugging Face

# Préparation des données pour RAGAS

data = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": []  # Tu peux remplir avec des réponses de référence ou laisser vide
}

for item in df.to_dict(orient="records"):
    data["question"].append(item["question"])
    data["answer"].append(item["answer"])
    data["contexts"].append(item["chunks"])  # ✅ ici on utilise "chunks"
    data["ground_truth"].append("")  # ou une réponse de référence si tu en as

ragas_dataset = Dataset.from_dict(data)


In [ ]:
# Évaluation avec RAGAS en utilisant le df construit grâce à la boucle de test

from ragas import evaluate
from ragas.metrics import context_precision, faithfulness, answer_relevancy

result = evaluate(
    ragas_dataset,
    metrics=[context_precision, faithfulness, answer_relevancy]
)

result.to_pandas()


Evaluating:   2%|▏         | 1/54 [00:01<01:39,  1.89s/it]Exception raised in Job[11]: IndexError(list index out of range)
Exception raised in Job[5]: IndexError(list index out of range)
Evaluating:   4%|▎         | 2/54 [00:04<01:55,  2.22s/it]Exception raised in Job[2]: IndexError(list index out of range)
Exception raised in Job[8]: IndexError(list index out of range)
Evaluating: 100%|██████████| 54/54 [03:12<00:00,  3.57s/it]


,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy
0,Pourquoi certaines personnes sont autistes ?,[enfants de mères exposées au virus de la rubé...,Salut ! C'est une question très intéressante e...,,0.200000,0.750000,NaN
1,Est-ce que l'autisme se guérit ?,[Utiliser les méthodes naturelles et biomédica...,L'autisme n'est pas quelque chose que l'on peu...,,0.250000,0.615385,NaN
2,Comment savoir si je suis autiste ?,"[Es-tu autiste, toi aussi ? Cherches-tu à mieu...",Savoir si tu es autiste peut être un chemin pe...,,0.000000,1.000000,NaN
3,Est-ce ma faute si je suis autiste ?,"[Es-tu autiste, toi aussi ? Cherches-tu à mieu...","Non, ce n'est absolument pas ta faute si tu es...",,0.000000,1.000000,NaN
4,Comment gérer mes émotions quand je suis en co...,[suivante vous donne d’autres astuces pour sur...,"Gérer ses émotions, surtout la colère, peut êt...",,0.000000,0.928571,NaN
5,Comment faire des amis quand on est autiste ?,[TOP 10 DES STRATÉGIES D’AMITIÉ Choisis le typ...,Faire des amis quand on est autiste peut sembl...,,1.000000,0.857143,0.961933
6,Pourquoi j'ai du mal à comprendre les expressi...,"[De même, j’ai remarqué que certains enfants a...",C'est tout à fait normal d'avoir du mal à comp...,,1.000000,0.705882,0.000000
7,Comment expliquer à mes amis que je suis autis...,[Vous n’avez pas à inventer des explications p...,Expliquer à tes amis que tu es autiste peut se...,,0.500000,0.947368,0.979367
8,Que faire si je me sens submergé par les bruit...,[341 Sensibilité sensorielle Cela peut mener à...,Lorsque tu te sens submergé par les bruits ou ...,,1.000000,0.947368,NaN
9,Est-ce que je suis fou ?,[On m’a prise pour une extraterrestre et un ph...,"Salut,\n\nTu n'es pas fou. Se sentir différent...",,0.000000,0.923077,0.000000


Interpretation: 

Ce tableau présente des scores entre 0 et 1 pour chaque métrique:

    - context_precision : les chunks sont-ils vraiment utiles ?

    - faithfulness : la réponse respecte-t-elle le contenu des chunks ?

    - answer_relevancy : la réponse est-elle bien alignée avec la question ?

Il nous permet de détecter les cas où le modèle hallucine, dévie du ton bienveillant, ou utilise des sources peu pertinentes.

Analyse des résultats obtenus:

📊 1. context_precision

Définition : Quelle proportion des chunks récupérés sont réellement utiles pour répondre à la question.

0.00 → 0.25 : Les chunks sont probablement hors sujet ou peu pertinents.

1.00 : Tous les chunks sont directement utiles à la réponse.

🔍 Ce qu'on observe :
Certaines questions ont une précision faible → chunks peu pertinents.

D'autres ont une précision parfaite → excellente récupération.

💡 Interprétation : Notre index FAISS fonctionne bien pour certaines questions, mais d’autres semblent mal matcher. 

On pourrait :

    - augmenter la granularité du chunking,

    - enrichir les métadonnées pour filtrer par thème,

    - tester une autre méthode d’embedding.

🧠 2. faithfulness

Définition : Est-ce que la réponse générée respecte le contenu des chunks ? (pas d’hallucination)

1.00 : La réponse est parfaitement fidèle aux sources.

< 0.70 : Risque de déviation ou d’invention.

🔍 Ce qu'on observe :

Les réponses sont globalement fidèles (souvent > 0.75).

Même quand les chunks sont peu pertinents, le LLM reste prudent.

💡 Interprétation : Notre prompt bienveillant et structuré joue un rôle clé ici. Nous pouvons être confiant sur la sécurité des réponses, même si les sources sont faibles.

🎯 3. answer_relevancy
Définition : Est-ce que la réponse répond bien à la question posée ?

1.00 : Réponse parfaitement pertinente.

0.00 : Réponse hors sujet ou vague.

🔍 Ce qu'on observe :

Cette métrique est NaN pour les lignes sans reference (ground truth).

    - Score élevé → réponse jugée pertinente.

    - Score nul → réponse jugée hors sujet (selon la référence).

💡 Interprétation : Pour que cette métrique soit calculée, nous devons fournir une reference (réponse de référence) dans ton dataset. Même quelques lignes bien calibrées suffisent pour enrichir l’analyse.

🧭 Pour la suite, nous allons ajouter des références  (ground truth) afin de mieux évaluer la métrique 'answer relevancy'. Pour ce faire, nous allons ajouter la colonne 'référence' qui sert à indiquer ce que serait une réponse idéale à la question posée.



In [29]:
# Réponses de référence pour comparaison

references = [
    "L’autisme est une condition neurodéveloppementale liée à des différences dans le fonctionnement du cerveau. Ce n’est ni une faute, ni une maladie.",
    "L’autisme ne se guérit pas, car ce n’est pas une maladie. On peut cependant apprendre à mieux vivre avec, grâce à des soutiens adaptés.",
    "Pour savoir si tu es autiste, il est important de consulter des professionnels spécialisés. Ils prendront en compte ton vécu et ton fonctionnement.",
    "Non, ce n’est absolument pas ta faute. L’autisme fait partie de qui tu es, et tu mérites d’être compris et respecté.",
    "Quand tu es en colère, tu peux essayer de respirer lentement, t’isoler un moment ou parler à un adulte de confiance. Il existe des stratégies pour t’aider.",
    "Faire des amis peut être difficile, mais possible. Cherche des personnes bienveillantes, sois toi-même et prends ton temps.",
    "Les expressions faciales peuvent être complexes à décoder. Tu peux apprendre à les reconnaître avec des outils ou des exemples concrets.",
    "Tu peux expliquer à tes amis que l’autisme influence ta manière de penser et de ressentir. S’ils sont ouverts, ils comprendront mieux qui tu es.",
    "Si tu te sens submergé, tu peux utiliser des bouchons d’oreilles, des lunettes filtrantes ou t’éloigner un moment. C’est normal de chercher des solutions.",
    "Non, tu n’es pas fou. Ressentir des choses différemment ne veut pas dire que quelque chose ne va pas. Tu as le droit d’être toi.",
    "Pour gérer le stress au lycée, tu peux organiser ton temps, parler à un adulte ou utiliser des techniques de relaxation. Tu n’es pas seul.",
    "Pour mieux dormir, essaie de garder une routine, d’éviter les écrans avant le coucher et de créer un environnement calme.",
    "Tu n’es pas seul à avoir du mal à te faire des amis. Cela peut prendre du temps, mais il existe des moyens pour créer des liens.",
    "La peur des autres peut venir de l’anxiété ou d’expériences passées. Tu peux apprendre à la surmonter avec des petits pas et du soutien.",
    "Tu n’es pas bizarre. Tu es unique, comme tout le monde. L’autisme ne te rend pas moins digne d’amitié ou de respect.",
    "Pour mieux t’organiser, tu peux utiliser des listes, des rappels visuels ou des routines. Trouve ce qui fonctionne pour toi.",
    "L’humour peut être subtil ou culturel. Tu peux apprendre à le décoder avec des exemples ou en demandant des explications.",
    "Pour gérer le stress des contrôles, prépare-toi à l’avance, repose-toi bien et parle de tes inquiétudes à un adulte. Tu as le droit d’être soutenu."
]


In [32]:
# Vérification de la correspondance des références
df[["question", "answer", "reference"]].head()


,question,answer,reference
0,Pourquoi certaines personnes sont autistes ?,Salut ! C'est une question très intéressante e...,L’autisme est un trouble neurodéveloppemental ...
1,Est-ce que l'autisme se guérit ?,L'autisme n'est pas quelque chose que l'on peu...,"L’autisme n’est pas une maladie à guérir, mais..."
2,Comment savoir si je suis autiste ?,Savoir si tu es autiste peut être un chemin pe...,Se poser cette question est déjà important. Un...
3,Est-ce ma faute si je suis autiste ?,"Non, ce n'est absolument pas ta faute si tu es...","Non, ce n’est jamais la faute de la personne. ..."
4,Comment gérer mes émotions quand je suis en co...,"Gérer ses émotions, surtout la colère, peut êt...","Gérer ses émotions, surtout la colère, peut êt..."


In [33]:
# Préparation des données pour RAGAS avec les réponses de référence
from datasets import Dataset

data = {
    "question": df["question"].tolist(),
    "answer": df["answer"].tolist(),
    "contexts": df["chunks"].tolist(),
    "ground_truth": df["reference"].tolist()
}

ragas_dataset = Dataset.from_dict(data)


In [ ]:
# Évaluation avec RAGAS en utilisant les réponses de référence
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness, answer_relevancy

result = evaluate(
    ragas_dataset,
    metrics=[context_precision, faithfulness, answer_relevancy]
)

result.to_pandas()


Evaluating:   2%|▏         | 1/54 [00:01<01:30,  1.70s/it]Exception raised in Job[11]: IndexError(list index out of range)
Exception raised in Job[14]: IndexError(list index out of range)
Exception raised in Job[2]: IndexError(list index out of range)
Exception raised in Job[8]: IndexError(list index out of range)
Evaluating:  43%|████▎     | 23/54 [01:25<01:18,  2.52s/it]Exception raised in Job[35]: IndexError(list index out of range)
Exception raised in Job[32]: IndexError(list index out of range)
Evaluating: 100%|██████████| 54/54 [03:12<00:00,  3.57s/it]


,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy
0,Pourquoi certaines personnes sont autistes ?,[enfants de mères exposées au virus de la rubé...,Salut ! C'est une question très intéressante e...,L’autisme est un trouble neurodéveloppemental ...,1.000000,0.687500,NaN
1,Est-ce que l'autisme se guérit ?,[Utiliser les méthodes naturelles et biomédica...,L'autisme n'est pas quelque chose que l'on peu...,"L’autisme n’est pas une maladie à guérir, mais...",1.000000,0.583333,NaN
2,Comment savoir si je suis autiste ?,"[Es-tu autiste, toi aussi ? Cherches-tu à mieu...",Savoir si tu es autiste peut être un chemin pe...,Se poser cette question est déjà important. Un...,0.638889,1.000000,NaN
3,Est-ce ma faute si je suis autiste ?,"[Es-tu autiste, toi aussi ? Cherches-tu à mieu...","Non, ce n'est absolument pas ta faute si tu es...","Non, ce n’est jamais la faute de la personne. ...",1.000000,1.000000,NaN
4,Comment gérer mes émotions quand je suis en co...,[suivante vous donne d’autres astuces pour sur...,"Gérer ses émotions, surtout la colère, peut êt...","Gérer ses émotions, surtout la colère, peut êt...",0.804167,1.000000,NaN
5,Comment faire des amis quand on est autiste ?,[TOP 10 DES STRATÉGIES D’AMITIÉ Choisis le typ...,Faire des amis quand on est autiste peut sembl...,Faire des amis peut prendre du temps. Il exist...,1.000000,0.857143,0.952178
6,Pourquoi j'ai du mal à comprendre les expressi...,"[De même, j’ai remarqué que certains enfants a...",C'est tout à fait normal d'avoir du mal à comp...,Les expressions faciales peuvent être difficil...,1.000000,1.000000,0.000000
7,Comment expliquer à mes amis que je suis autis...,[Vous n’avez pas à inventer des explications p...,Expliquer à tes amis que tu es autiste peut se...,Expliquer à tes amis que tu es autiste peut se...,1.000000,0.947368,NaN
8,Que faire si je me sens submergé par les bruit...,[341 Sensibilité sensorielle Cela peut mener à...,Lorsque tu te sens submergé par les bruits ou ...,Lorsque tu te sens submergé par les bruits ou ...,0.887500,0.950000,NaN
9,Est-ce que je suis fou ?,[On m’a prise pour une extraterrestre et un ph...,"Salut,\n\nTu n'es pas fou. Se sentir différent...",Tu n'es absolument pas fou. Se sentir différen...,0.755556,0.923077,NaN


Les NaN dans la colonne answer_relevancy sont un comportement connu dans RAGAS, et il y a plusieurs raisons possibles, ainsi que des solutions concrètes pour y remédier.

🧪 Pourquoi answer_relevancy peut être NaN

🔹 1. Réponse trop courte ou trop vague
Si la réponse générée est trop générique, le LLM n’arrive pas à “reverse-engineer” des questions pertinentes à partir de cette réponse — ce qui empêche le calcul de similarité.

🔹 2. Question trop ambiguë ou trop éloignée du contenu
Si la question ne correspond pas bien aux chunks récupérés, la réponse peut être hors sujet ou trop floue, ce qui perturbe la métrique.

🔹 3. Référence manquante ou mal alignée
Même si answer_relevancy est censée être reference-free, certains cas ambigus peuvent être mieux évalués si une ground_truth est fournie et bien alignée.

🔹 4. Problème technique ou modèle non compatible
Certains modèles d’embedding ou de LLM utilisés en arrière-plan peuvent ne pas générer les questions inverses correctement. Cela a été signalé dans ce fil GitHub

🧭 Ce qui fonctionne déjà très bien:

Fidélité : les scores de faithfulness sont élevés, ce qui montre que le modèle respecte les sources.

Bienveillance : le prompt est clair, doux, et inclusif. Les réponses générées sont rassurantes et jamais culpabilisantes.

Validation : on construit un jeu de test structuré, annoté, et tu utilises RAGAS pour identifier les points faibles sans dramatiser.

🔮 Ce qui viendra renforcer la pertinence

Ajouter des livres structurés et bien ciblés (comme Situations sociales ou des guides psychoéducatifs).

Affiner le chunking pour mieux capter les nuances (ex. : découpage par situation ou par conseil).

Enrichir les métadonnées pour filtrer par thème ou tonalité.

Tester des prompts alternatifs pour voir si certaines formulations améliorent la pertinence sans nuire à la bienveillance.